# Housing prices in Hyderabad, India

## Project Objective 🎯

The objective of this project is to develop a regression model to predict housing prices in Hyderabad, India. Using features such as the property's area, location, number of bedrooms, and available amenities, the model will aim to estimate the market value of a property as accurately as possible.

- This predictive model will be a valuable tool for:
- Home Buyers and Sellers: To obtain an objective price estimate for a property.
- Real Estate Agents: To assist with property valuation and client advisory.
- Investors: To identify potentially undervalued or overvalued properties in the market.

## 6. Deploy the best models

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error
import joblib
import sys
sys.path.append('../../src/utils')

# Utilities
from housing_prices_column_definition import Column

best_model_path = '../../datasets/processed/housing_prices/best_hyderabad_house_price_model.pkl'

cleanUpDataset = pd.read_parquet('../../datasets/processed/housing_prices/hyderabad_house_price_cleanup.parquet')

In [2]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.linear_model import LinearRegression

# --- 1. Definir los transformadores por tipo de columna ---

# Transformador para columnas numéricas ('AREA')
# 1. Aplica el logaritmo np.log1p
numeric_transformer = FunctionTransformer(np.log1p)

# Transformador para columnas categóricas ('LOCATION')
categorical_transformer = OneHotEncoder(
    # Reemplaza tu lógica de "threshold = 10" y "Other"
    min_frequency=10,
    # Codifica todo lo que tenga < 10 cuentas como una sola categoría
    handle_unknown='infrequent_if_exist',
    # Coincide con tu drop_first=True
    drop='first',
    # Para que funcione con el modelo lineal
    sparse_output=False
)

# --- 2. Combinar transformadores con ColumnTransformer ---
# Esto aplica el pipeline correcto a la columna correcta
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, [Column.AREA.name]),
        ('cat', categorical_transformer, [Column.LOCATION.name])
    ],
    # 'passthrough' mantiene columnas que no mencionamos    
    remainder='passthrough'
)

# --- 3. Crear el Pipeline de Modelo ---
# Este pipeline primero aplica el preprocesador (paso 2)
# y luego alimenta la salida al modelo.
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression(fit_intercept=True))
])

# --- 4. Envolver todo con TransformedTargetRegressor ---
# Esto maneja la transformación de 'PRICE' automáticamente
final_pipeline = TransformedTargetRegressor(
    regressor=model_pipeline,
    func=np.log1p,        # Función para aplicar a 'y' antes de entrenar
    inverse_func=np.expm1 # Función para aplicar a la predicción
)

training_label = cleanUpDataset[Column.PRICE.name]
training_features = cleanUpDataset.drop(Column.PRICE.name, axis=1)

final_pipeline.fit(training_features, training_label)

joblib.dump(final_pipeline, best_model_path)

['../../datasets/processed/housing_prices/best_hyderabad_house_price_model.pkl']